Получаем типы сервисов

In [1]:
from townsnet.provision.service_type import ServiceType, Category, AccessibilityType, SupplyType

SERVICE_TYPES = {
    21 : ServiceType(
      id=21, 
      accessibility_value=300, 
      supply_value=61,
      accessibility_type=AccessibilityType.METERS, 
      supply_type=SupplyType.CAPACITY_PER_1000, 
      category=Category.BASIC,
      weight=0.2
    ),
    63 : ServiceType(
      id=63, 
      accessibility_value=2000,
      supply_value=105.2, 
      accessibility_type=AccessibilityType.METERS, 
      supply_type=SupplyType.CAPACITY_PER_1000, 
      category=Category.ADDITIONAL,
      weight=0.3
    ),
    28 : ServiceType(
      id=28, 
      accessibility_value=10,
      supply_value=13, 
      accessibility_type=AccessibilityType.MINUTES, 
      supply_type=SupplyType.CAPACITY_PER_1000, 
      category=Category.COMFORT,
      weight=0.3
    ),
}

In [2]:
SERVICE_TYPES_NAMES = {
  21: 'kindergarten',
  63: 'cinema',
  28: 'polyclinic',
}

In [3]:
import geopandas as gpd

provisions = {SERVICE_TYPES[st_id] : gpd.read_file(f'./data/res/{st_name}_towns.geojson') for st_id, st_name in SERVICE_TYPES_NAMES.items()}

In [5]:
from townsnet.provision.social_model import SocialModel

towns_gdf = list(provisions.values())[0].copy()
sm = SocialModel(towns_gdf, provisions)

In [6]:
project_gdf = gpd.read_file('./data/territories/shlis.geojson')
project_gdf

,building,name,addr:street,addr:housenumber,addr:city,type,is_living,geometry
0,None,None,None,None,None,project,False,"POLYGON ((30.99391 59.92460, 30.99972 59.92939..."


In [8]:
eval = sm.evaluate_provisions(project_gdf.iloc[0].geometry)

In [30]:
import pandas as pd
eval_df = pd.DataFrame([{
  'id' : st.id,
  'weight': st.weight,
  'category': st.category.name,
  'provision': prov
  } for st, prov in eval.items()]).set_index('id', drop=True)
eval_df = eval_df[eval_df.provision >= 0.1]
eval_df

,weight,category,provision
id,,,
28,0.3,COMFORT,0.116279


In [31]:
additional, basic, comfort = eval_df.groupby('category').agg({'weight':'sum'})['weight']

ValueError: not enough values to unpack (expected 3, got 1)